In [35]:
import requests
import numpy as np
import pandas as pd
import json
from IPython.display import display
import re
import os,sys,inspect
currentdir = os.path.dirname(os.path.abspath(inspect.getfile(inspect.currentframe())))
parentdir = os.path.dirname(currentdir)
sys.path.insert(0,parentdir)

from lib.twitter_keys import keys

In [36]:
sql_select = '''
select * from tweets where lang = 'en'
'''


In [53]:
import psycopg2 as pg2
import psycopg2.extras as pgex
this_host='54.69.228.16'
this_user='postgres'
this_password='postgres'
conn = pg2.connect(host = this_host, 
                        user = this_user,
                        password = this_password)
cur = conn.cursor(cursor_factory=pgex.RealDictCursor)
#cur.execute(sql_create)
#cur.execute(sql_drop)
#cur.execute(sql_insert)
#conn.commit()
cur.execute(sql_select)
rows = cur.fetchall()
conn.close()
df = pd.DataFrame(rows)

In [54]:
len(df)

62583

In [39]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.decomposition import TruncatedSVD
from sklearn.pipeline import Pipeline

In [40]:
import numpy as np
import pandas as pd
import seaborn as sns
from sklearn.decomposition import TruncatedSVD
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import LabelEncoder
import matplotlib.pyplot as plt

In [ ]:
import pandas as pd
import numpy as np
from IPython import display
from bs4 import BeautifulSoup
import requests
!pip install nltk
import nltk

import re
from spacy.en import STOP_WORDS
from spacy.en import English
nlp = English()

In [ ]:
def cleaner(text):
    emoji_pattern = re.compile("["
        u"\U0001F600-\U0001F64F"  # emoticons
        u"\U0001F300-\U0001F5FF"  # symbols & pictographs
        u"\U0001F680-\U0001F6FF"  # transport & map symbols
        u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
                           "]+", flags=re.UNICODE)
    text = re.sub(emoji_pattern, '', text)
    text = text.lower()
    text = re.sub(r'http\S+', '',text)
    text = re.sub(r'#\S+', '',text)
    text = re.sub(r'@\S+', '',text)
    text = re.sub('/(?<!\S).(?!\S)\s*/',' ',text)
    text = re.sub('\s+',' ',text)
    text = ' '.join([i.lemma_ for i in nlp(text) 
                   if i.orth_ not in STOP_WORDS])
    text = re.sub('\n',' ',text)

    
    return text

In [49]:
df[['tweet_content','cleaned_tweet','location','created_at']].tail(10)

,tweet_content,cleaned_tweet,location,created_at
52847,caught this lil guy in my font yard!! @ tha su...,caught this lil guy in my font yard!! @ tha su...,"norwalk, ca",Thu Jun 01 21:37:56 +0000 2017
52848,i'm so sick that i'm tired but i'm too tired t...,i'm so sick that i'm tired but i'm too tired t...,"los angeles, ca",Thu Jun 01 21:37:57 +0000 2017
52849,maybe this is just an attempt to compete with ...,maybe this is just an attempt to compete with ...,"los angeles, ca",Thu Jun 01 21:37:58 +0000 2017
52850,today i got to visit my favorite presidents li...,today i got to visit my favorite presidents li...,"simi valley, ca",Thu Jun 01 21:37:59 +0000 2017
52851,crazy lines (@ costco gasoline in marina del r...,crazy lines (@ costco gasoline in marina del r...,"culver city, ca",Thu Jun 01 21:37:59 +0000 2017
52852,day 1️⃣ of #popshipchallenge is any variation ...,day 1️⃣ of is any variation of the 💖 - my idea...,"los angeles, ca",Thu Jun 01 21:38:01 +0000 2017
52853,quit that job u hate and do what u love. it's ...,quit that job u hate and do what u love. it's ...,"los angeles, ca",Thu Jun 01 21:38:02 +0000 2017
52854,@amc_tv are showing the day after tomorrow at ...,are showing the day after tomorrow at the mom...,"los angeles, ca",Thu Jun 01 21:38:02 +0000 2017
52855,high powr revoluccion obama,high powr revoluccion obama,"south gate, ca",Thu Jun 01 21:38:03 +0000 2017
52856,its fucking pride month!!! so lets see whose d...,its fucking pride month!!! so lets see whose d...,"santa fe springs, ca",Thu Jun 01 21:38:03 +0000 2017


In [50]:
df[['tweet_content','cleaned_tweet','location','created_at']][df['tweet_content'].str.contains("trump")].tail(100)

,tweet_content,cleaned_tweet,location,created_at
51051,@iankenyonnfl how does he change anything by s...,how does he change anything by staying? trump...,"el segundo, ca",Thu Jun 01 21:14:00 +0000 2017
51057,@alexjonessucks @shicks_sam @chlanandria @vame...,hey carlos why do dumbocrat…,"california, usa",Thu Jun 01 21:14:08 +0000 2017
51059,trump pulls out of paris climate accord and pu...,trump pulls out of paris climate accord and pu...,"los angeles, ca",Thu Jun 01 21:14:11 +0000 2017
51064,trump whaiit u gotta say bout ur name getting ...,trump whaiit u gotta say bout ur name getting ...,"south gate, ca",Thu Jun 01 21:14:12 +0000 2017
51089,i wish trumps dad had pulled out!! #fucktrump,i wish trumps dad had pulled out!!,"los angeles, ca",Thu Jun 01 21:14:36 +0000 2017
51094,this is my favorite delusional #trump meme. gr...,this is my favorite delusional meme. greatest ...,"la verne, ca",Thu Jun 01 21:14:40 +0000 2017
51115,rice tuiehlps radio trump villa radio go threw it,rice tuiehlps radio trump villa radio go threw it,"south gate, ca",Thu Jun 01 21:14:55 +0000 2017
51123,@stationcdrkelly @realdonaldtrump thoughtful c...,thoughtful comments from someone way smarter ...,"los angeles, ca",Thu Jun 01 21:15:06 +0000 2017
51127,there is only one person with the actual power...,there is only one person with the actual power...,"los angeles, ca",Thu Jun 01 21:15:09 +0000 2017
51130,@aynrandpaulryan @realdonaldtrump you are just...,you are just as sick as kathy griffin. go to ...,"california, usa",Thu Jun 01 21:15:12 +0000 2017


In [22]:
df[['screen_name','tweet_content','cleaned_tweet','location','latitude','longitude','created_at','time_zone']].head(20)

,screen_name,tweet_content,cleaned_tweet,location,latitude,longitude,created_at,time_zone
0,tmj_lax_hrta,"Interested in a #job in #Glendale, CA? This co...",Interested in a in CA? This could be a great f...,"California, USA",34.144045,-118.257472,Wed May 31 23:50:46 +0000 2017,Eastern Time (US & Canada)
1,mejiaaa_,i'm such a weak ass pu$$y bitch,I'm such a weak ass pu$$y bitch,"los angeles, ca",0.000000,0.000000,Wed May 31 23:54:24 +0000 2017,pacific time (us & canada)
2,Revision_Health,remember to get up and move today! move free l...,Remember to get up and move today! Move Free L...,"beverly hills, ca",0.000000,0.000000,Wed May 31 23:54:24 +0000 2017,None
3,Franciscojruizm,taking a quick break from preparing charts to ...,Taking a quick break from preparing charts to ...,"los angeles, ca",34.052200,-118.243000,Wed May 31 23:54:25 +0000 2017,central time (us & canada)
4,gemarble357,don't let #hardball fool you. william f buckle...,Don't let fool you. William F Buckley Jr was a...,"los angeles, ca",0.000000,0.000000,Wed May 31 23:54:25 +0000 2017,america/los_angeles
5,LidiaKarinaaa,i wonder what his reaction to chivas winning 😂...,I wonder what his reaction to chivas Winning 😂,"los angeles, ca",0.000000,0.000000,Wed May 31 23:54:27 +0000 2017,None
6,MsLynnsWorld,glad today was wednesday already (@ oaktree ca...,Glad today was Wednesday already (@ Oaktree Ca...,"los angeles, ca",34.052622,-118.251809,Wed May 31 23:54:27 +0000 2017,pacific time (us & canada)
7,tmj_sjc_retail,join the kohl's corporation team! see our late...,Join the Kohl's Corporation team! See our late...,"california, usa",37.763025,-121.239744,Wed May 31 23:54:28 +0000 2017,eastern time (us & canada)
8,ramiromendes,the #energy encompassed within hip- hop is bey...,The encompassed within hip- hop is beyond the,"los angeles, ca",0.000000,0.000000,Wed May 31 23:54:29 +0000 2017,pacific time (us & canada)
9,BillCool,@collinsdw lol. sorry. you being from texas an...,Lol. Sorry. You being from Texas and all... I...,"los angeles, ca",0.000000,0.000000,Wed May 31 23:54:30 +0000 2017,america/los_angeles


In [25]:
df[['screen_name','tweet_content','cleaned_tweet','location','created_at','time_zone']].tail(40)

,screen_name,tweet_content,cleaned_tweet,location,created_at,time_zone
50352,FoodnessG,guys...so we all know ice cream can have a few...,guys...so we all know ice cream can have a few...,"long beach, ca",Thu Jun 01 21:04:43 +0000 2017,pacific time (us & canada)
50353,daniellecuccio,"keep shining, keep smiling thursday. 🌞🌟✨ this ...","keep shining, keep smiling thursday. 🌞🌟✨ this ...","los angeles, ca",Thu Jun 01 21:04:44 +0000 2017,pacific time (us & canada)
50354,ideasbychuck,half photo of me n half drawing of me by somyo...,half photo of me n half drawing of me by somyo...,"los angeles, ca",Thu Jun 01 21:04:44 +0000 2017,quito
50355,auntiejodie,i miss spy magazine.,i miss spy magazine.,"long beach, ca",Thu Jun 01 21:04:47 +0000 2017,america/los_angeles
50356,RachaelTyrell_,@tycobb06071984 @hakasematt @ramzpaul @youtube...,she said my argument is like a child saying i...,"west hollywood, ca",Thu Jun 01 21:04:47 +0000 2017,america/los_angeles
50357,MylesGordon9,confession: i don't really 'get' modern art @ ...,confession: i don't really 'get' modern art @ ...,"los angeles, ca",Thu Jun 01 21:04:48 +0000 2017,eastern time (us & canada)
50358,adriannn_cp3,saw a side of me that i never knew,saw a side of me that i never knew,"south gate, ca",Thu Jun 01 21:04:49 +0000 2017,None
50359,ptto911,"@sarahjeong he has idiot in his name, what do ...","he has idiot in his name, what do you expect ...","fullerton, ca",Thu Jun 01 21:04:49 +0000 2017,pacific time (us & canada)
50360,lettuceshoppe,digging this all mint wall at the @museumicecr...,digging this all mint wall at the 💚if you have...,"los angeles, ca",Thu Jun 01 21:04:50 +0000 2017,pacific time (us & canada)
50361,SusanDoyle1963,@sethabramson i remember,i remember,"los angeles, ca",Thu Jun 01 21:04:51 +0000 2017,None


In [ ]:
for i in range(100):
    df['cleaned_tweet'][i] = cleaner(df['tweet_content'][i])
    #print(df[['tweet_content','cleaned_tweet']][i])

In [ ]:
df['tweet_content'][0]

In [ ]:
cleaner(df['tweet_content'][0])

In [ ]:
df['cleaned_tweet'][1]

In [ ]:
df[['tweet_content','cleaned_tweet']]

In [ ]:
df[['tweet_content','cleaned_tweet']]

In [ ]:
tfidf_vectorizer = TfidfVectorizer(min_df = 3, stop_words = 'english')

In [ ]:
from sklearn.feature_extraction.text import 

In [ ]:
cv = CountVectorizer()